<a href="https://colab.research.google.com/github/MehrdadJannesar/CNN_AdaBoost/blob/master/ResNet50_laitec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense,Input,BatchNormalization
from keras.applications.resnet50 import ResNet50
from keras.datasets import mnist
import numpy as np

Using TensorFlow backend.


In [2]:
#Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()
X_train = train_images.astype('float32')
X_test = test_images.astype('float32')

X_train /= 255
X_test /= 255
N_classes = 10
Y_train = keras.utils.to_categorical(train_labels,N_classes)
Y_test = keras.utils.to_categorical(test_labels,N_classes)

170500096/170498071 [==============================] - 6s 0us/step


In [4]:
#Create model
input_layer = Input(shape=(32,32,3))
resnet = ResNet50(include_top = False, pooling = 'avg'
                  , weights = 'imagenet')(input_layer)

dense = Dense(N_classes, activation = 'softmax')(resnet)

myModel = Model(input_layer, dense)














94658560/94653016 [==============================] - 3s 0us/step



In [6]:
myModel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [5]:
input_ress = myModel.get_layer(index=1).inputs
output_ress = myModel.get_layer(index=1).outputs
ress_model = Model(inputs=input_ress,outputs=output_ress)

ress_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [7]:
sgd = keras.optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
myModel.compile(optimizer = sgd, loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

In [11]:
import datetime

start = datetime.datetime.now()

network = myModel.fit(X_train, Y_train, batch_size=128, epochs=10 )

end = datetime.datetime.now()

print('time compile is : ', str(end - start))

Epoch 1/10
50000/50000 [==============================] - 27s 549us/step - loss: 0.2305 - acc: 0.9216
Epoch 2/10
50000/50000 [==============================] - 28s 551us/step - loss: 0.1722 - acc: 0.9409
Epoch 3/10
50000/50000 [==============================] - 28s 551us/step - loss: 0.1418 - acc: 0.9520
Epoch 4/10
50000/50000 [==============================] - 27s 550us/step - loss: 0.1132 - acc: 0.9627
Epoch 5/10
50000/50000 [==============================] - 28s 551us/step - loss: 0.0902 - acc: 0.9707
Epoch 6/10
50000/50000 [==============================] - 28s 551us/step - loss: 0.0872 - acc: 0.9715
Epoch 7/10
50000/50000 [==============================] - 27s 549us/step - loss: 0.0766 - acc: 0.9755
Epoch 8/10
50000/50000 [==============================] - 28s 551us/step - loss: 0.0603 - acc: 0.9810
Epoch 9/10
50000/50000 [==============================] - 28s 556us/step - loss: 0.0611 - acc: 0.9807
Epoch 10/10
50000/50000 [==============================] - 28s 551us/step - loss: 